# New Sagemaker Train,Deploy models tutorial


- 07/15/21

# Step 2. Prepare the data


In [1]:
# import libraries
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
from IPython.display import display
from time import gmtime, strftime
from sagemaker.predictor import csv_serializer

# Define IAM role
role = get_execution_role()
prefix = 'sagemaker/DEMO-xgboost-dm'
my_region = boto3.session.Session().region_name # set the region of the instance

# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
xgboost_container = sagemaker.image_uris.retrieve("xgboost", my_region, "latest")

print("Success - the MySageMakerInstance is in  " + my_region + " region. You will use the " + xgboost_container + " container for your SageMaker endpoint.")

Success - the MySageMakerInstance is in  us-east-1 region. You will use the 811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest container for your SageMaker endpoint.


In [2]:
#reate the S3 bucket to store your data. 
bucket_name = 'my-demo-bucket-for-022221ft' # <--- CHANGE THIS VARIABLE TO A UNIQUE NAME FOR YOUR BUCKET
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-1':
      s3.create_bucket(Bucket=bucket_name)
    else: 
      s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 bucket created successfully


In [3]:
#Download the data to your SageMaker instance and load as df
try:
  urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
  print('Success: downloaded bank_clean.csv.')
except Exception as e:
  print('Data load error: ',e)

try:
  model_data = pd.read_csv('./bank_clean.csv',index_col=0)
  print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


In [4]:
# 70/30 train test split
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


In [5]:
## Load the data into the s3 bucket
pd.concat([train_data['y_yes'], 
           train_data.drop(['y_no', 'y_yes'], axis=1)],axis=1)\
                            .to_csv('train.csv', index=False, header=False)

boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

# Step 3. Train the ML model

In [6]:
## Set up sagemaker session and make an xgboost estimator
sess = sagemaker.Session()
xgb = sagemaker.estimator.Estimator(xgboost_container,role, instance_count=1,
                                    instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket_name, prefix),
                                    sagemaker_session=sess)

xgb.set_hyperparameters(max_depth=5,eta=0.2,
                        gamma=4,min_child_weight=6,
                        subsample=0.8,silent=0,
                        objective='binary:logistic',num_round=100)

In [7]:
xgb.fit({'train': s3_input_train})

2021-07-15 14:54:52 Starting - Starting the training job...
2021-07-15 14:55:14 Starting - Launching requested ML instancesProfilerReport-1626360891: InProgress
......
2021-07-15 14:56:15 Starting - Preparing the instances for training.........
2021-07-15 14:57:39 Downloading - Downloading input data...
2021-07-15 14:58:15 Training - Downloading the training image..Arguments: train
[2021-07-15:14:58:28:INFO] Running standalone xgboost training.
[2021-07-15:14:58:28:INFO] Path /opt/ml/input/data/validation does not exist!
[2021-07-15:14:58:28:INFO] File size need to be processed in the node: 3.38mb. Available memory size in the node: 8374.37mb
[2021-07-15:14:58:28:INFO] Determined delimiter of CSV input is ','
[14:58:28] S3DistributionType set as FullyReplicated
[14:58:28] 28831x59 matrix with 1701029 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[14:58:28] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 14 pruned nodes, max_

# Step 4. Deploy the model

In [8]:
## Deploy the model
xgb_predictor = xgb.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')


-------------------!

In [9]:
## Get predictions 
from sagemaker.serializers import CSVSerializer

test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
xgb_predictor.serializer = CSVSerializer() # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

(12357,)


# Step 5. Evaluate model performance

In [10]:
## Evaluate model performance 
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))



Overall Classification Rate: 89.5%

Predicted      No Purchase    Purchase
Observed
No Purchase    90% (10769)    37% (167)
Purchase        10% (1133)     63% (288) 



# Step 6. Clean up

In [11]:
#Delete your endpoint:
xgb_predictor.delete_endpoint(delete_endpoint_config=True)


In [12]:
# Delete your bucket
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

[{'ResponseMetadata': {'RequestId': '8QK0GQKWZWDTA4Z1',
   'HostId': '6JGjjV8eWfA5p5mLvU8Kj3UXdlb2baZPOcN0CbCzlcuC2gx7eyYnmpYaQIYH+STsmaedhlUMwJs=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': '6JGjjV8eWfA5p5mLvU8Kj3UXdlb2baZPOcN0CbCzlcuC2gx7eyYnmpYaQIYH+STsmaedhlUMwJs=',
    'x-amz-request-id': '8QK0GQKWZWDTA4Z1',
    'date': 'Thu, 15 Jul 2021 15:09:13 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'sagemaker/DEMO-xgboost-dm/output/xgboost-2021-07-15-14-54-51-911/profiler-output/system/training_job_end.ts'},
   {'Key': 'sagemaker/DEMO-xgboost-dm/output/xgboost-2021-07-15-14-54-51-911/output/model.tar.gz'},
   {'Key': 'sagemaker/DEMO-xgboost-dm/output/xgboost-2021-07-15-14-54-51-911/rule-output/ProfilerReport-1626360891/profiler-output/profiler-reports/StepOutlier.json'},
   {'Key': 'sagemaker/DEMO-xgboost-dm/output/xgboost-2021-07-15-14-5